In [1]:
import numpy as np
import pandas as pd
import scipy.ndimage as ndi
import matplotlib.pyplot as plt

import os
import re
import time
import pyautogui
import matplotlib

# import skimage
from skimage import io, filters, exposure, measure, transform
from scipy.signal import find_peaks, savgol_filter
from scipy.optimize import linear_sum_assignment

pd.set_option('mode.chained_assignment',None)

# %matplotlib widget 
# %matplotlib inline
%matplotlib qt
# matplotlib.use('Qt5Agg')
matplotlib.rcParams.update({'figure.autolayout': True})

SMALLER_SIZE = 8
SMALL_SIZE = 12
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALLER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

dateFormatExcel = re.compile('\d{2}/\d{2}/\d{4}')
dateFormatOk = re.compile('\d{2}-\d{2}-\d{2}')

In [2]:
SCALE_100X = 15.8 # pix/µm 

## Classes & functions

### Import exp table

In [3]:
# experimentalDataDir = "C://Users//josep//Desktop//ActinCortexAnalysis//ExperimentalData"
experimentalDataDir = "C://Users//JosephVermeil//Desktop//ActinCortexAnalysis//ExperimentalData"

def getExperimentalConditions(save = False):
    # Getting the table
    experimentalDataFile = 'ExperimentalConditions.csv'
    experimentalDataFilePath = os.path.join(experimentalDataDir, experimentalDataFile)
    expConditionsDF = pd.read_csv(experimentalDataFilePath, sep=';',header=0)
    print('Extracted a table with ' + str(expConditionsDF.shape[0]) + ' lines and ' + str(expConditionsDF.shape[1]) + ' columns.')
    
    # Cleaning the table
    try:
        for c in expConditionsDF.columns:
            if 'Unnamed' in c:
                expConditionsDF = expConditionsDF.drop([c], axis=1)
            if '.1' in c:
                expConditionsDF = expConditionsDF.drop([c], axis=1)
        expConditionsDF = expConditionsDF.convert_dtypes()

        listTextColumns = []
        for col in expConditionsDF.columns:
            try:
                if expConditionsDF[col].dtype == 'string':
                    listTextColumns.append(col)
            except:
                aaaa=0
                #Ok

        expConditionsDF[listTextColumns] = expConditionsDF[listTextColumns].apply(lambda x: x.str.replace(',','.'))

        expConditionsDF['scale pixel per um'] = expConditionsDF['scale pixel per um'].astype(float)
        try:
            expConditionsDF['optical index correction'] = \
                      expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[0]).astype(float) \
                    / expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[1]).astype(float)
        except:
            print('optical index correction already in ' + str(expConditionsDF['optical index correction'].dtype) + ' type.')

        expConditionsDF['magnetic field correction'] = expConditionsDF['magnetic field correction'].astype(float)
        expConditionsDF['with fluo images'] = expConditionsDF['with fluo images'].astype(bool)

        try:
            expConditionsDF['ramp field'] = \
            expConditionsDF['ramp field'].apply(lambda x: [x.split(';')[0], x.split(';')[1]] if not pd.isnull(x) else [])
        except:
            aaaa=0
            #Ok

        dateExemple = expConditionsDF.loc[expConditionsDF.index[1],'date']

        if re.match(dateFormatExcel, dateExemple):
            print('dates corrected')
            expConditionsDF.loc[1:,'date'] = expConditionsDF.loc[1:,'date'].apply(lambda x: x.split('/')[0] + '-' + x.split('/')[1] + '-' + x.split('/')[2][2:])        
        
    except:
        print('Unexpected bug with the cleaning step')

    if save:
        saveName = 'ExperimentalConditions.csv'
        savePath = os.path.join(experimentalDataDir, saveName)
        expConditionsDF.to_csv(savePath, sep=';')

    expConditionsDF['manipID'] = expConditionsDF['date'] + '_' + expConditionsDF['manip']
#     reorgaList = np.array([i for i in range(len(expConditionsDF.columns))])
#     reorgaList[2] = reorgaList[-1]
#     reorgaList[3:] = reorgaList[3:] - np.ones(len(reorgaList)-3)
#     expConditionsDF = expConditionsDF[expConditionsDF.columns[reorgaList]]
    
    return(expConditionsDF)

expDf = getExperimentalConditions()

expDf

Extracted a table with 40 lines and 24 columns.
optical index correction already in Float64 type.
dates corrected


,date,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,...,beads bright spot delta,normal field,ramp field,compression duration,with fluo images,normal field multi images,loop structure,bacteria,comments,manipID
0,DEFAULT,DEFAULT,DEFAULT,DEFAULT,DEFAULT,100X,15.8,oil,0.875,1.15,...,0,5,3_40,1s,False,3,<NA>,<NA>,<NA>,DEFAULT_DEFAULT
1,20-08-04,M1,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,0,10,3_40,1s,False,3,<NA>,<NA>,<NA>,20-08-04_M1
2,20-08-04,M2,compressions and constant field,doxycyclin,BSA coated glass,100X,15.8,oil,0.875,1.15,...,0,10,3_40,1s,False,3,<NA>,<NA>,<NA>,20-08-04_M2
3,20-08-05,M1,compressions and constant field,doxycyclin,BSA coated glass,100X,15.8,oil,0.875,1.15,...,0,10,3_40,1s,False,3,<NA>,<NA>,<NA>,20-08-05_M1
4,20-08-05,M2,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,0,10,3_40,1s,False,3,<NA>,<NA>,<NA>,20-08-05_M2
5,20-08-07,M1,compressions and constant field,doxycyclin,BSA coated glass,100X,15.8,oil,0.875,1.15,...,0,10,3_40,1s,False,3,<NA>,<NA>,<NA>,20-08-07_M1
6,20-08-07,M2,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,0,10,3_40,1s,False,3,<NA>,<NA>,<NA>,20-08-07_M2
7,21-01-18,M1,compressions,doxycyclin,20um fibronectin discs,100X,15.8,oil,0.875,1.23,...,0,5,3_40,1s,True,3,<NA>,<NA>,<NA>,21-01-18_M1
8,21-01-18,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.23,...,0,5,3_40,1s,True,3,<NA>,<NA>,<NA>,21-01-18_M2
9,21-01-18,M3,compressions,doxycyclin,20um fibronectin discs,100X,15.8,oil,0.875,1.23,...,0,5,3_40,1s,True,3,<NA>,<NA>,<NA>,21-01-18_M3


### Utility functions

In [4]:
def findInfosInFileName(f, infoType):
    if infoType in ['M', 'P', 'C']:
        acceptedChar = [str(i) for i in range(10)] + ['.', '-']
        string = '_' + infoType
        iStart = re.search(string, f).end()
        i = iStart
        infoString = '' + f[i]
        while f[i+1] in acceptedChar and i < len(f)-1:
            i += 1
            infoString += f[i]
    
    elif infoType == 'manipID':
        datePos = re.search(r"[\d]{1,2}-[\d]{1,2}-[\d]{2}", f)
        date = f[datePos.start():datePos.end()]
        manip = 'M' + findInfosInFileName(f, 'M')
        infoString = date + '_' + manip
        
    elif infoType == 'cellID':
        datePos = re.search(r"[\d]{1,2}-[\d]{1,2}-[\d]{2}", f)
        date = f[datePos.start():datePos.end()]
        infoString = date + '_' + 'M' + findInfosInFileName(f, 'M') + \
                            '_' + 'P' + findInfosInFileName(f, 'P') + \
                            '_' + 'C' + findInfosInFileName(f, 'C')
    
    return(infoString)

def isFileOfInterest(f, manips, wells, cells):
    test = False
    if f.endswith(".tif"):
        if manips == 'all':
            test = True
        else:
            try:
                manips_str = [str(i) for i in manips]
            except:
                manips_str = [str(manips)]
            infoM = findInfosInFileName(f, 'M')
            if infoM in manips_str:
                if wells == 'all':
                    test = True
                else:
                    try:
                        wells_str = [str(i) for i in wells]
                    except:
                        wells_str = [str(wells)]
                    infoP = findInfosInFileName(f, 'P')
                    if infoP in wells_str:
                        if cells == 'all':
                            test = True
                        else:
                            try:
                                cells_str = [str(i) for i in cells]
                            except:
                                cells_str = [str(cells)]
                            infoC = findInfosInFileName(f, 'C')
                            if infoC in cells_str:
                                test = True
    return(test)

def compute_cost_matrix(XY1,XY2):
    N1, N2 = XY1.shape[0],XY2.shape[0]
    M = np.zeros((N1, N2))
    for i in range(N1):
        for j in range(N2):
            M[i,j] = (np.sum((XY2[j,:] - XY1[i,:]) ** 2))
    return(M)

def ui2array(uixy):
    n = len(uixy)
    A = np.zeros((n, 2))
    for i in range(n):
        A[i,0], A[i,1] = uixy[i][0], uixy[i][1]
    return(A)

def getROI(roiSize, x0, y0, nx, ny):
    roiSize += roiSize%2
    x1 = int(np.floor(x0) - roiSize*0.5) - 1
    x2 = int(np.floor(x0) + roiSize*0.5)
    y1 = int(np.floor(y0) - roiSize*0.5) - 1
    y2 = int(np.floor(y0) + roiSize*0.5)
    if min([x1,nx-x2,y1,ny-y2]) < 0:
        validROI = False
    else:
        validROI = True
    return(x1, y1, x2, y2, validROI)

In [5]:
# Test utility functions

# a = 'aa12-32-12AZF34_M2'
# datePos = re.search(r"[\d]{1,2}-[\d]{1,2}-[\d]{2}", a)
# a[datePos.start():datePos.end()]
# findInfosInFileName(a, 'manipID')
# a = 'aaa_bbb'
# a.split('_')
# s = '21-04-27_M1_P1_C4_R40_disc20um_wFluo'
# findInfosInFileName(s, 'cellID')
# [(331.26256664936864, 258.949997248057), (403.98986101188945, 258.9821671859072)]

# XY1 = np.array([[1,2],[4,2],[7,2]])
# XY2 = np.array([[4.2,1.9],[7.3,2.4],[10.5,2.9]])
# M = compute_cost_matrix(XY1,XY2)
# row_ind, col_ind = linear_sum_assignment(M)
# for i in range(len(row_ind)):
#     e1 = XY1[row_ind[i]]
#     e2 = XY2[col_ind[i]]
#     print(str(e1) + ' match with ' + str(e2))
    
# XY1 = np.array([[1,2]])
# XY2 = np.array([[4.2,1.9],[7.3,2.4],[10.5,2.9]])
# M = compute_cost_matrix(XY1,XY2)
# row_ind, col_ind = linear_sum_assignment(M)
# for i in range(len(row_ind)):
#     e1 = XY1[row_ind[i]]
#     e2 = XY2[col_ind[i]]
#     print(str(e1) + ' match with ' + str(e2))
# print(M[0])

### Tracker Classes

In [6]:
class PincherTimeLapse:
    
    def __init__(self, I, cellID, manipDict, NB = 2, ):
        nS, ny, nx = I.shape[0], I.shape[1], I.shape[2]
        self.I = I
        self.threshold = 0
        self.NB = NB
        self.nx = nx
        self.ny = ny
        self.nS = nS
        self.listFrames = []
        self.listTrajectories = []
        self.dictLog = {'Slice' : np.array([i+1 for i in range(nS)]), \
                        'Status' : np.zeros(nS, dtype = int), \
                        # in the status field: -1 means excluded ; 0 means ramp ; >0 means position in the n-uplet
                        'Status_2' : np.zeros(nS, dtype = int), \
                        # in the status_2 field: -1 means excluded ; 0 means ramp ; >0 means number of the n-uplet
#                         'Fluo' : np.zeros(nS, dtype = bool), \
#                         'Black' : np.zeros(nS, dtype = bool), \
                        'UI' : np.zeros(nS, dtype = bool), \
                        'UILog' : np.array(['' for i in range(nS)], dtype = '<U8'), \
                        'UIxy' : np.zeros((nS,NB,2), dtype = int)}
        
        self.detectBeadsResult = pd.DataFrame(
            {'Area' : [], 'StdDev' : [], 'XM' : [], 'YM' : [], 'Slice' : []})
        
        self.cellID = cellID
        self.expType = manipDict['experimentType']
        self.wFluo = bool(manipDict['with fluo images'])
        
        loopStruct = manipDict['loop structure'].split('_')
        self.loop_totalSize = int(loopStruct[0])
        if self.expType == 'compressions':
            self.loop_rampSize = int(loopStruct[1])
        else:
            self.loop_rampSize = 0
        if len(loopStruct) == 3: # This 3rd part of the 'loopStruct' field is the nb of frames at the end
        # of each loop which are not part of the main analysis and should be excluded. Typically fluo images.
            self.loop_excludedSize = int(loopStruct[2])
        else:
            self.loop_excludedSize = 0
        self.nLoop = int(np.round(nS/self.loop_totalSize))
        self.blackFramesPerLoop = np.zeros(self.nLoop)
        self.Nuplet = manipDict['normal field multi images']
        
    def checkIfBlackFrames(self):
        for i in range(self.nLoop):
            j = ((i+1)*self.loop_totalSize) - 1
            checkSum = np.sum(self.I[j])
            while checkSum == 0:
#                 self.dictLog['Black'][j] = True
                self.dictLog['Status'][j] = -1
                self.dictLog['Status_2'][j] = -1
                self.blackFramesPerLoop[i] += 1
                j -= 1
                checkSum = np.sum(self.I[j])
              
    def saveFluoAside(self, fluoDirPath = ''):
        if self.wFluo:
#             if not os.path.exists(fluoDirPath):
#                 os.makedirs(fluoDirPath)
            for i in range(self.nLoop):
                j = int(((i+1)*self.loop_totalSize) - 1 - self.blackFramesPerLoop[i])
#                 self.dictLog['Fluo'][j] = True
                self.dictLog['Status'][j] = -1
                self.dictLog['Status_2'][j] = -1
                
    def determineFramesStatus(self):
        N0 = self.loop_totalSize
        Nramp0 = self.loop_rampSize
        Nexclu = self.loop_excludedSize
        nUp = self.Nuplet
        N = N0 - Nexclu
        Nct = N - Nramp0
        i_nUp = 1
        for i in range(self.nLoop):
            jstart = int(i*N0)
            if Nramp0 == 0:
                for j in range(N):
                    self.dictLog['Status'][jstart + j] = 1 + j%self.Nuplet
                    self.dictLog['Status_2'][j] = i_nUp + j//self.Nuplet
            else:
                Nramp = Nramp0-self.blackFramesPerLoop[i]
                for j in range(Nct//2):
                    self.dictLog['Status'][jstart + j] = 1 + j%self.Nuplet
                    self.dictLog['Status_2'][jstart + j] = i_nUp + j//self.Nuplet
                i_nUp = max(self.dictLog['Status_2']) + 1
                jstart += int(Nct//2 + Nramp)
                for j in range(Nct//2):
                    self.dictLog['Status'][jstart + j] = 1 + j%self.Nuplet
                    self.dictLog['Status_2'][jstart + j] = i_nUp + j//self.Nuplet
                i_nUp = max(self.dictLog['Status_2']) + 1
                
    def saveLog(self, display = 1, save = False, path = ''):
        dL = {}
        dL['Slice'], dL['Status'], dL['Status_2'] = \
            self.dictLog['Slice'], self.dictLog['Status'], self.dictLog['Status_2']
#         dL['Fluo'], dL['Black'] = \
#             self.dictLog['Fluo'], self.dictLog['Black']
        dL['UI'], dL['UILog'] = \
            self.dictLog['UI'], self.dictLog['UILog']
        for i in range(self.NB):
            dL['UIx'+str(i+1)] = self.dictLog['UIxy'][:,i,0]
            dL['UIy'+str(i+1)] = self.dictLog['UIxy'][:,i,1]
        dfLog = pd.DataFrame(dL)
        if display == 1:
            print('\n\n* Initialized Log Table:\n')
            print(dfLog)
        if display == 2:
            print('\n\n* Filled Log Table:\n')
            print(dfLog[dfLog['UI']])
        if save:
            dfLog.to_csv(path, sep='\t')
        
    def importLog(self, path):
        dfLog = pd.read_csv(path, sep='\t')
        dL = dfLog.to_dict()
        self.dictLog['Slice'], self.dictLog['Status'], self.dictLog['Status_2'] = \
            dfLog['Slice'].values, dfLog['Status'].values, dfLog['Status_2'].values
#         self.dictLog['Fluo'], self.dictLog['Black'] = \
#             dfLog['Fluo'].values, dfLog['Black'].values
        self.dictLog['UI'], self.dictLog['UILog'] = \
            dfLog['UI'].values, dfLog['UILog'].values
        for i in range(self.NB):
            xkey, ykey = 'UIx'+str(i+1), 'UIy'+str(i+1)
            self.dictLog['UIxy'][:,i,0] = dfLog[xkey].values
            self.dictLog['UIxy'][:,i,1] = dfLog[ykey].values
        
    def computeThreshold(self, method = 'otsu'):
        # TBC
#         factorT = 0.8*(self.D == 4.5) + 0.6*(self.D == 2.7)
        factorT = 0.35
        threshold = factorT*filters.threshold_otsu(self.I)
        self.threshold = threshold
        
    def testThresholding(self):
        I_test = self.I[self.nS//2]
        I_thresh = I_test > self.threshold
        fig, ax = plt.subplots(1,1)
        ax.imshow(I_thresh, cmap = 'gray')
        fig.show()
        
    def makeFramesList(self):
        for i in range(self.nS):
            status = self.dictLog['Status'][i]
            status_2 = self.dictLog['Status_2'][i]
            if self.dictLog['Status'][i] >= 0:
                self.listFrames.append(Frame(self.I[i], i, self.NB, self.threshold, self.Nuplet, status, status_2))
    
    def detectBeads(self, resFileImported, display = False):
        for frame in self.listFrames: #[:3]:
            if not resFileImported:
                frame.detectBeads()
                self.detectBeadsResult = pd.concat([self.detectBeadsResult, frame.resDf])
            else:
                resDf = self.detectBeadsResult.loc[self.detectBeadsResult['Slice'] == frame.iS+1]
                frame.resDf = resDf
                
            frame.makeListBeads()
            
        if not resFileImported:
            self.detectBeadsResult = self.detectBeadsResult.convert_dtypes()
            self.detectBeadsResult.reset_index(inplace=True)
            self.detectBeadsResult.drop(['index'], axis = 1, inplace=True)
        
        if display:
            print('\n\n* Detected Beads Result:\n')
            print(self.detectBeadsResult)

        
#     def makeBeadsDetectResult(self, save=False, path=''):
#         df = pd.DataFrame({'Area' : [], 'StdDev' : [], 'XM' : [], 'YM' : [], 'Slice' : []})
#         df = pd.concat([pd.DataFrame(frame.resDict) for frame in self.listFrames])
#         df = df.reset_index()
#         df = df.drop(['index'], axis = 1) # To be checked
#         if save:
#             df.to_csv(path, sep=';')
#         return(df)

    def saveBeadsDetectResult(self, path=''):
        self.detectBeadsResult.to_csv(path, sep='\t')
    
    def importBeadsDetectResult(self, path=''):
        df = pd.read_csv(path, sep='\t')
        for c in df.columns:
            if 'Unnamed' in c:
                df.drop([c], axis = 1, inplace=True)
        self.detectBeadsResult = df
        
    def findBestStd_V0(self):
        """
        This ugly function is my best attempt to implement sth very simple in a robust way.
        In the 'status' field, -1 means excluded image, 0 means image that isn't part of a N-uplet of images, and k>0 means position in the N-uplet of images.
        For each image in the N-uplet, I want to reconsititute this N-uplet (meaning the list of Nuplet consecutive images numbered from 1 to Nuplet, minus the images eventually with no beads detected).
        Then for each N-uplet of images, i want to find the max standard deviation and report its position because it's for the max std that the X and Y detection is the most precise.
        An exemple: with these inputs:
        Nuplet = 3
        status = [1,2,3,0,0,0,1,2, 3, 1, 2, 3, 1, 2]
            iS = [0,1,2,3,4,5,6,7,11,12,13,14,15,16]
           std = [1,5,9,5,5,5,1,5, 8, 2, 6, 9, 2, 6]
        The function will return bestStd, a list of boolean with the same length.
        Where status = 0, bestStd = True (the image is not part of a N-uplet, thus it need to be analysed regardless of its std).
        Where satus > 0, the function will cut the lists in N-uplet of max size 3:
        status -> [1,2,3] ; [1,2] ; [ 3] ; [ 1, 2, 3] ; [ 1, 2]
            iS -> [0,1,2] ; [6,7] ; [11] ; [12,13,14] ; [15,16]
           std -> [1,5,9] ; [1,5] ; [ 8] ; [ 2, 6, 9] ; [ 2, 6]
             i -> [0,1,2] ; [6,7] ; [ 8] ; [ 9,10,11] ; [12,13]
        and then will find the best std in each of those fragment and put a True at that position (list 'i') in bestStd, so in this example, at i = 2, 7, 8, 11 ,13
        So the output is: bestStd = [False, False, True, True, True, True, False, True, True, False, False, True, False, True]
        """
        
        Nuplet = self.Nuplet
        status = self.listTrajectories[0].dict['Status']
        iS = self.listTrajectories[0].dict['iS']
        nT = self.listTrajectories[0].nT
        std = np.zeros(nT)
        for i in range(0,self.NB):
            std += np.array(self.listTrajectories[i].dict['StdDev'])
        
        bestStd = np.zeros(nT, dtype = bool)

        current_Nup_status = []
        current_Nup_iS = []
        current_Nup_std = []
        current_Nup_i = []
        for i in range(nT):
            if status[i] == 0:
                bestStd[i] = True
            else:
                if len(current_Nup_i) == 0:
                    current_Nup_status = [status[i]]
                    current_Nup_iS = [iS[i]]
                    current_Nup_std = [std[i]]
                    current_Nup_i = [i]
                else:
                    if status[i] > current_Nup_status[-1] and (iS[i]-current_Nup_iS[-1]) < Nuplet:
                        current_Nup_status.append(status[i])
                        current_Nup_iS.append(iS[i])
                        current_Nup_std.append(std[i])
                        current_Nup_i.append(i)
                    else:
    #                     print(current_Nup_status, current_Nup_iS, current_Nup_std, current_Nup_i)
                        i_bestStdInCurrentNuplet = int(np.argmax(np.array(current_Nup_std)))
    #                     print(i_bestStdInCurrentNuplet)
                        i_bestStd = int(current_Nup_i[i_bestStdInCurrentNuplet])
                        bestStd[i_bestStd] = True
                        # then:
                        current_Nup_status = [status[i]]
                        current_Nup_iS = [iS[i]]
                        current_Nup_std = [std[i]]
                        current_Nup_i = [i]

        # Need to do it one last time at the end
        i_bestStdInCurrentNuplet = int(np.argmax(np.array(current_Nup_std)))
        i_bestStd = int(current_Nup_i[i_bestStdInCurrentNuplet])
        bestStd[i_bestStd] = True

        return(bestStd)
    
    def findBestStd(self): # To be improved
        """
        Better than findBestStd_V0
        """
        
        Nuplet = self.Nuplet
        nT = self.listTrajectories[0].nT
        status_2 = self.listTrajectories[0].dict['Status_2']
        std = np.zeros(nT)
        for i in range(self.NB):
            std += np.array(self.listTrajectories[i].dict['StdDev'])
        
        bestStd = np.zeros(nT, dtype = bool)
        i = 0
        while i < nT:
            if status_2[i] == 0:
                bestStd[i] = True
                i += 1
            elif status_2[i] > 0:
                s2 = status_2[i]
                L = [i]
                j = 0
                while i+j < nT-1 and status_2[i+j+1] == s2: # Evaluation paresseuse des booleens
                    j += 1
                    L.append(i+j)
                print(L)    
                loc_std = std[L]
                i_bestStd = i + int(np.argmax(loc_std))
                bestStd[i_bestStd] = True
                L = []
                i = i + j + 1

        return(bestStd)
        
    def buildTrajectories(self):
        # NB: 'iF': index in a list of Frames ; 'iB': index in list of Beads or Trajectories ; 'iS': index of the slice in the image I
        # Boi refers to the 'Beads of interest', ie the beads that are being tracked
        init_iF = 0
        init_ok = False
        while not init_ok:
            init_iS = self.listFrames[init_iF].iS
            if not self.dictLog['UI'][init_iS]:
                self.listFrames[init_iF].show()
                mngr = plt.get_current_fig_manager()
                mngr.window.setGeometry(700,100,1000, 800)
                QA = pyautogui.confirm(
                    text='Can you point the beads of interest\nin the image ' + str(init_iS + 1) + '?',
                    title='Initialise tracker', 
                    buttons=['Yes', 'Next Frame', 'Quit'])
                if QA == 'Yes':
                    init_ok = True
                    ui = plt.ginput(2, timeout=0)
                    uiXY = ui2array(ui)
                    fig = plt.gcf()
                    plt.close(fig)
                    self.dictLog['UI'][init_iS] = True
                    self.dictLog['UILog'][init_iS] = 'init_' + QA
                    self.dictLog['UIxy'][init_iS] = uiXY
                elif QA == 'Next Frame':
                    fig = plt.gcf()
                    plt.close(fig)
                    self.dictLog['UI'][init_iS] = True
                    self.dictLog['UILog'][init_iS] = 'init_' + QA
                    init_iF += 1
                else:
                    fig = plt.gcf()
                    plt.close(fig)
                    return('Bug')
            else:
                QA = self.dictLog['UILog'][init_iS]
                if QA == 'init_Yes':
                    init_ok = True
                    uiXY = self.dictLog['UIxy'][init_iS]
                elif QA == 'Next Frame':
                    init_iF += 1
                else:
                    aa
        
        init_BXY = self.listFrames[init_iF].beadsXYarray()
        M = compute_cost_matrix(uiXY,init_BXY)
        row_ind, col_ind = linear_sum_assignment(M) # row_ind -> clicks / col_ind -> listBeads
        # Sort the initial beads to have them ordered by growing x coord.
        sortM = np.array([[init_BXY[col_ind[i],0], col_ind[i]] for i in range(len(col_ind))])
        sortM = sortM[sortM[:, 0].argsort()]
        init_iBoi = sortM[:, 1].astype(int)
        init_BoiXY = np.array([init_BXY[col_ind[i]] for i in range(len(col_ind))])
        
        for iB in range(self.NB):
            self.listTrajectories.append(Trajectory(self.I))
#             self.listTrajectories[iB].seriesBeads.append(self.listFrames[init_iF].listBeads[col_ind[iB]])
#             self.listTrajectories[iB].pointerBeads.append(init_iBoi[iB])
#             self.listTrajectories[iB].series_iS.append(self.listFrames[init_iF].iS)
#             self.listTrajectories[iB].seriesXY.append(init_BoiXY[iB].tolist())
#             self.listTrajectories[iB].current_iS = self.listFrames[init_iF].iS
            #
            self.listTrajectories[iB].dict['Bead'].append(self.listFrames[init_iF].listBeads[init_iBoi[iB]])
            self.listTrajectories[iB].dict['iF'].append(init_iF)
            self.listTrajectories[iB].dict['iS'].append(self.listFrames[init_iF].iS)
            self.listTrajectories[iB].dict['iB'].append(init_iBoi[iB])
            self.listTrajectories[iB].dict['X'].append(init_BoiXY[iB][0])
            self.listTrajectories[iB].dict['Y'].append(init_BoiXY[iB][1])
            self.listTrajectories[iB].dict['StdDev'].append(self.listFrames[init_iF].beadsStdDevarray()[init_iBoi[iB]])
            self.listTrajectories[iB].dict['Status'].append(self.listFrames[init_iF].status)
            self.listTrajectories[iB].dict['Status_2'].append(self.listFrames[init_iF].status_2)
            self.listTrajectories[iB].dict['idxCompression'].append(1 * (self.listFrames[init_iF].status == 0))

        previous_iF = init_iF
        previous_iBoi = init_iBoi
        previous_BXY = init_BXY
        previous_BoiXY = init_BoiXY
        
        for iF in range(init_iF+1, len(self.listFrames)):
            validFrame = True
            
            if len(self.listFrames[iF].listBeads) < self.NB:
                validFrame = False
            
            else:
                BXY = self.listFrames[iF].beadsXYarray()
                M = compute_cost_matrix(previous_BXY,BXY)
                row_ind, col_ind = linear_sum_assignment(M)
                costs = [M[row_ind[iB],col_ind[iB]] for iB in range(len(row_ind))]
                
                # Assess wether the algo should aks for user input
                askUI = False
                if (max(costs)**0.5) * (1/SCALE_100X) > 1: # If the greatest distance travelled by any object is greater than 1um
                    askUI = True

                if not askUI: # Automatically asign the positions of the next beads
                    iBoi = [col_ind[row_ind.tolist().index(iB)] for iB in previous_iBoi]
                    BoiXY = np.array([BXY[iB] for iB in iBoi])

                elif askUI: # Ask user input to asign the positions of the next beads
                    iS = self.listFrames[iF].iS
                    # Case 1: the UI has not been previously saved in the dictLog
                    # Then ask for UI and save it in the dictLog
                    if not self.dictLog['UI'][iS]:
                        # Display the image, plot beads positions and current trajectories & ask the question
                        self.listFrames[iF].show()
                        for iB in range(self.NB):
                            T = self.listTrajectories[iB]
                            ax = plt.gca()
                            T.plot(ax, iB)
                        mngr = plt.get_current_fig_manager()
                        mngr.window.setGeometry(700,100,1000, 800)
                        QA = pyautogui.confirm(
                            text='Can you point the beads of interest\nin the image ' + str(iS + 1) + '?',
                            title='', 
                            buttons=['No', 'Yes', 'Abort Mission!'])
                        
                        # According to the question's answer:
                        if QA == 'Yes':
                            ui = plt.ginput(2, timeout=0)
                            uiXY = ui2array(ui)
                            fig = plt.gcf()
                            plt.close(fig)
                            self.dictLog['UI'][iS] = True
                            self.dictLog['UILog'][iS] = QA
                            print(QA)
                            print(self.dictLog['UILog'][iS])
                            self.dictLog['UIxy'][iS] = uiXY
                        elif QA == 'No':
                            validFrame = False
                            fig = plt.gcf()
                            plt.close(fig)
                            self.dictLog['UI'][iS] = True
                            self.dictLog['UILog'][iS] = QA
                            print(QA)
                            print(self.dictLog['UILog'][iS])
                        elif QA == 'Abort Mission!':
                            validFrame = False
                            fig = plt.gcf()
                            plt.close(fig)
                            return('Bug')
                    
                    # Case 2: the UI has been previously saved in the dictLog.
                    # Then just import the previous answer from the dictLog
                    else:
                        QA = self.dictLog['UILog'][iS]
                        if QA == 'Yes':
                            uiXY = self.dictLog['UIxy'][iS]
                            
                        elif QA == 'No':
                            validFrame = False
                            fig = plt.gcf()
                            plt.close(fig)
            
            # If there were more than NB objects, and then the QA wasn't 'No', the frame is valid.
            if validFrame:
                M = compute_cost_matrix(uiXY,BXY)
                row_ind, col_ind = linear_sum_assignment(M)
                sortM = np.array([[BXY[col_ind[i],0], col_ind[i]] for i in range(len(col_ind))])
                sortM = sortM[sortM[:, 0].argsort()]
                iBoi = sortM[:, 1].astype(int)
                BoiXY = np.array([BXY[iB] for iB in iBoi])
                # idxCompression = 0 if not in a ramp, and = number of ramp else. Basically increase by 1 each time you have an interval between two ramps.
                idxCompression = (self.listFrames[iF].status == 0) * (max(self.listTrajectories[iB].dict['idxCompression']) + 1 * (self.listTrajectories[iB].dict['idxCompression'][-1] == 0))
                for iB in range(self.NB):
#                     self.listTrajectories[iB].seriesBeads.append(self.listFrames[iF].listBeads[iBoi[iB]])
#                     self.listTrajectories[iB].pointerBeads.append(iBoi[iB])
#                     self.listTrajectories[iB].series_iS.append(self.listFrames[iF].iS)
#                     self.listTrajectories[iB].seriesXY.append(BoiXY[iB].tolist())
#                     self.listTrajectories[iB].current_iS = self.listFrames[iF].iS
                    #
                    self.listTrajectories[iB].dict['Bead'].append(self.listFrames[iF].listBeads[iBoi[iB]])
                    self.listTrajectories[iB].dict['iF'].append(iF)
                    self.listTrajectories[iB].dict['iS'].append(self.listFrames[iF].iS)
                    self.listTrajectories[iB].dict['iB'].append(iBoi[iB])
                    self.listTrajectories[iB].dict['X'].append(BoiXY[iB][0])
                    self.listTrajectories[iB].dict['Y'].append(BoiXY[iB][1])
                    self.listTrajectories[iB].dict['StdDev'].append(self.listFrames[iF].beadsStdDevarray()[iBoi[iB]])
                    self.listTrajectories[iB].dict['Status'].append(self.listFrames[iF].status)
                    self.listTrajectories[iB].dict['Status_2'].append(self.listFrames[iF].status_2)
                    self.listTrajectories[iB].dict['idxCompression'].append(idxCompression)
                    
                previous_iF = iF
                previous_iBoi = iBoi
                previous_BXY = BXY
                previous_BoiXY = BoiXY
                
        for iB in range(self.NB):
            for k in self.listTrajectories[iB].dict.keys():
                self.listTrajectories[iB].dict[k] = np.array(self.listTrajectories[iB].dict[k])
                
        # Now we have a functional Trajectory object
        # Time to refine it
        
        nT = len(self.listTrajectories[0].dict['Bead'])
        
        # (a) Add the pointer to the correct line of the _Field.txt file.
        # It's just exactly the iS already saved in the dict, except if there are black images at the end of loops.
        # In that case you have to skip the X lines corresponding to the end of the ramp part, X being the nb of black images at the end of the current loop
        # This is because when black images occurs, they do because of the high frame rate during ramp parts and thus replace these last ramp images.
        
        for iB in range(self.NB):
            self.listTrajectories[iB].nT = nT
            iField = []
            for i in range(nT):
                S = self.listTrajectories[iB].dict['iS'][i]
                iLoop = (S//self.loop_totalSize)
                offset = self.blackFramesPerLoop[iLoop]
                i_lim = iLoop*self.loop_totalSize + (self.loop_totalSize - ((self.loop_totalSize-self.loop_rampSize)//2) - (self.loop_excludedSize + offset))
                # i_lim is the first index after the end of the ramp
                addOffset = (S >= i_lim)
                SField = S + int(addOffset*offset)
                iField.append(SField)
            self.listTrajectories[iB].dict['iField'] = iField
            
        # (b) Find the image with the best std within each n-uplet
            
        bestStd = self.findBestStd()
        
        for i in range(self.NB):
            self.listTrajectories[i].dict['bestStd'] = bestStd

In [7]:
# Test findBestStd

# def findBestStd(Nuplet, status, iS, std):
#     """
#     This ugly function is my best attempt to implement sth very simple in a robust way.
#     In the 'status' field, -1 means excluded image, 0 means image that isn't part of a N-uplet of images, and k>0 means position in the N-uplet of images.
#     For each image in the N-uplet, I want to reconsititute this N-uplet (meaning the list of Nuplet consecutive images numbered from 1 to Nuplet, minus the images eventually with no beads detected).
#     Then for each N-uplet of images, i want to find the max standard deviation and report its position because it's for the max std that the X and Y detection is the most precise.
#     An exemple: with these inputs:
#     Nuplet = 3
#     status = [1,2,3,0,0,0,1,2, 3, 1, 2, 3, 1, 2]
#         iS = [0,1,2,3,4,5,6,7,11,12,13,14,15,16]
#        std = [1,5,9,5,5,5,1,5, 8, 2, 6, 9, 2, 6]
#     The function will return bestStd, a list of boolean with the same length.
#     Where status = 0, bestStd = True (the image is not part of a N-uplet, thus it need to be analysed regardless of its std).
#     Where satus > 0, the function will cut the lists in N-uplet of max size 3:
#     status -> [1,2,3] ; [1,2] ; [ 3] ; [ 1, 2, 3] ; [ 1, 2]
#         iS -> [0,1,2] ; [6,7] ; [11] ; [12,13,14] ; [15,16]
#        std -> [1,5,9] ; [1,5] ; [ 8] ; [ 2, 6, 9] ; [ 2, 6]
#          i -> [0,1,2] ; [6,7] ; [ 8] ; [ 9,10,11] ; [12,13]
#     and then will find the best std in each of those fragment and put a True at that position (list 'i') in bestStd, so in this example, at i = 2, 7, 8, 11 ,13
#     So the output is: bestStd = [False, False, True, True, True, True, False, True, True, False, False, True, False, True]
#     """
    
#     Ntraj = len(iS)
#     bestStd = np.zeros(Ntraj, dtype = bool)
#     M = np.array([status, iS, std])
#     current_Nup_status = []
#     current_Nup_iS = []
#     current_Nup_std = []
#     current_Nup_i = []
#     for i in range(Ntraj):
#         if status[i] == 0:
#             bestStd[i] = True
#         else:
#             if len(current_Nup_i) == 0:
#                 current_Nup_status = [status[i]]
#                 current_Nup_iS = [iS[i]]
#                 current_Nup_std = [std[i]]
#                 current_Nup_i = [i]
#             else:
#                 if status[i] > current_Nup_status[-1] and (iS[i]-current_Nup_iS[-1]) < Nuplet:
#                     current_Nup_status.append(status[i])
#                     current_Nup_iS.append(iS[i])
#                     current_Nup_std.append(std[i])
#                     current_Nup_i.append(i)
#                 else:
# #                     print(current_Nup_status, current_Nup_iS, current_Nup_std, current_Nup_i)
#                     i_bestStdInCurrentNuplet = int(np.argmax(np.array(current_Nup_std)))
# #                     print(i_bestStdInCurrentNuplet)
#                     i_bestStd = int(current_Nup_i[i_bestStdInCurrentNuplet])
#                     bestStd[i_bestStd] = True
#                     # then:
#                     current_Nup_status = [status[i]]
#                     current_Nup_iS = [iS[i]]
#                     current_Nup_std = [std[i]]
#                     current_Nup_i = [i]
                    
#     # Need to do it one last time at the end
#     i_bestStdInCurrentNuplet = int(np.argmax(np.array(current_Nup_std)))
#     i_bestStd = int(current_Nup_i[i_bestStdInCurrentNuplet])
#     bestStd[i_bestStd] = True
    
#     return(bestStd)

# Nuplet = 3
# status = [1,2,3,0,0,0,1,2, 3, 1, 2, 3, 1, 2]
# iS = [0,1,2,3,4,5,6,7,11,12,13,14,15,16]
# std = [1,5,9,5,5,5,1,5, 8, 2, 6, 9, 2, 6]
# bs = findBestStd(Nuplet, status, iS, std)
# bs

In [8]:
class Frame:
    
    def __init__(self, F, iS, NB, threshold, Nup, status, status_2):
        ny, nx = F.shape[0], F.shape[1]
        self.F = F # Note : Frame.F points directly to the i-th frame of the image I ! To have 2 different versions one should use np.copy(F)
        self.threshold = threshold
        self.NBoi = NB
        self.NBdetected = 0
        self.nx = nx
        self.ny = ny
        self.iS = iS
        self.listBeads = []
        self.trajPoint = []
        self.Nuplet = Nup
        self.status = status
        self.status_2 = status_2
        self.resDf = pd.DataFrame({'Area' : [], 'StdDev' : [], 'XM' : [], 'YM' : [], 'Slice' : []})

        
    def __str__(self):
        text = 'a'
        return(text)
    
    def show(self, strech = True):
        fig, ax = plt.subplots(1,1)
#         fig_size = plt.gcf().get_size_inches()
#         fig.set_size_inches(2 * fig_size)
        if strech:
            pStart, pStop = np.percentile(self.F, (1, 99))
            ax.imshow(self.F, cmap = 'gray', vmin = pStart, vmax = pStop)
        else:
            ax.imshow(self.F, cmap = 'gray')
        if len(self.listBeads) > 0:
            for B in self.listBeads:
                ax.plot([B.x], [B.y], c='orange', marker='o')
        fig.show()
    
    def detectBeads(self):
        F_bin = self.F > self.threshold
        F_lab, nObj = ndi.label(F_bin)
        props = measure.regionprops(F_lab)
        listValidLabels = []
        areas = np.zeros(nObj+1)
        for k in range(1, nObj+1):
            try:
                bb = props[k-1].bbox
                Valid = not (min(bb) == 0 or bb[2] == self.ny or bb[3] == self.nx) # Remove objects touching the edges of the frame

    #             # OPTION 1 - Compute the metrics on the filled shape ; NB: takes a lot of time
    #             F_fh = ndi.binary_fill_holes((F_lab == k).astype(int))
    #             tmp_props = measure.regionprops(F_fh.astype(int))
    #             A = tmp_props[0].area
    #             P = tmp_props[0].perimeter
    #             Circ = (4 * np.pi * A) / (P * P)
    #             Valid = Valid and A >= 100 and Circ >= 0.75 # Area and circularity criterion

                # OPTION 2 - Lower the criterion in circularity - NB: less selective
                A = props[k-1].area
                P = props[k-1].perimeter
                Circ = (4 * np.pi * A) / (P * P)
                Valid = Valid and A >= 100 and Circ >= 0.65 # Area and circularity criterion

            except:
                Valid = False
            if Valid:
                listValidLabels.append(k)
                areas[k] = A
                
#         F_labValid, nObjValid = ndi.label(F_lab)    
#         fig, ax = plt.subplots(1,1)
#         ax.imshow(F_labValid)
#         fig.show()
        
        centoids = np.array([ndi.center_of_mass(self.F, labels=F_lab, index=i) for i in listValidLabels])
        resDict = {}
        resDict['Area'] = np.array([areas[i] for i in listValidLabels]).astype(int)
        resDict['StdDev'] = np.array([ndi.standard_deviation(self.F, labels=F_lab, index=i) for i in listValidLabels])
        resDict['XM'] = centoids[:,1]
        resDict['YM'] = centoids[:,0]
        resDict['Slice'] = np.array([self.iS+1 for i in listValidLabels]).astype(int)
        self.resDf = pd.DataFrame(resDict)
        
#         print(self.resDict)

    def makeListBeads(self):
        self.NBdetected = self.resDf.shape[0]
        for i in range(self.NBdetected):
            d = {}
            for c in self.resDf.columns:
                d[c] = self.resDf[c].values[i]
            self.listBeads.append(Bead(d, self.F))
            
    def beadsXYarray(self):
        A = np.zeros((len(self.listBeads), 2))
        for i in range(len(self.listBeads)):
            b = self.listBeads[i]
            A[i,0], A[i,1] = b.x, b.y
        return(A)
    
    def beadsStdDevarray(self):
        A = np.zeros(len(self.listBeads))
        for i in range(len(self.listBeads)):
            b = self.listBeads[i]
            A[i] = b.std
        return(A)
    
    def detectDiameter(self, plot = 0):
        for i in range(len(self.listBeads)):
            B = self.listBeads[i]
            roughSize = np.floor(5*SCALE_100X)
            roughSize += roughSize%2
            x1_roughROI = max(int(np.floor(B.x) - roughSize*0.5) - 1, 0)
            x2_roughROI = min(int(np.floor(B.x) + roughSize*0.5), self.nx)
            y1_roughROI = max(int(np.floor(B.y) - roughSize*0.5) - 1, 0)
            y2_roughROI = min(int(np.floor(B.y) + roughSize*0.5), self.ny)
            
            F_roughRoi = self.F[y1_roughROI:y2_roughROI, x1_roughROI:x2_roughROI]   
            
            figh, axh = plt.subplots(1,1, figsize = (4,4))
            axh.hist(F_roughRoi.ravel(), bins=256, histtype='step', color='black')

            counts, binEdges = np.histogram(F_roughRoi.ravel(), bins=256)

            peaks, peaksProp = find_peaks(counts, height=100, threshold=None, distance=None, prominence=None, \
                               width=None, wlen=None, rel_height=0.5, plateau_size=None)
            
            peakThreshVal = 750
            
            if counts[peaks[0]] > peakThreshVal:
                B.D = 4.5
                
            else:
                B.D = 2.7
                
            if plot == 1:    
                print(B.D)
                B.show()
                
    def detectNeighbours(self, B):
        # To be completed
        return('')

In [9]:
class Bead:
    
    def __init__(self, d, F):
        self.x = d['XM']
        self.y = d['YM']
        self.D = 0
        self.area = d['Area']
        self.std = d['StdDev']
        self.iS = d['Slice']-1
        self.status = ''
        self.hasRightNeighbour = False
        self.hasLeftNeighbour = False
        self.F = F

    def show(self, strech = True):
        fig, ax = plt.subplots(1,1)
        if strech:
            pStart, pStop = np.percentile(self.F, (1, 99))
            ax.imshow(self.F, cmap = 'gray', vmin = pStart, vmax = pStop)
        else:
            ax.imshow(self.F, cmap = 'gray')
        ax.plot([self.x], [self.y], c='orange', marker='o')
        fig.show()

In [10]:
class Trajectory:
    def __init__(self, I):
        nS, ny, nx = I.shape[0], I.shape[1], I.shape[2]
        self.I = I
        self.nx = nx
        self.ny = ny
        self.nS = nS
        self.D = 0
        self.nT = 0
#         self.seriesBeads = []
#         self.pointerBeads = []
#         self.series_iS = []
#         self.seriesXY = []
#         self.current_iS = 0
        self.dict = {'X': [],'Y': [],'idxCompression': [],'StdDev': [], \
                     'Bead': [],'Status': [],'Status_2': [],'iF': [],'iS': [],'iB' : [],}
        # These columns will be added later : 'Neighbours', 'Z', 'bestStd' and 'iField'
        
    def __str__(self):
        text = 'iS : ' + str(self.series_iS)
        text += '\n'
        text += 'XY : ' + str(self.seriesXY)
        return(text)
    
    def to_df(self):
        df = pd.DataFrame(self.dict)
        return(df)
    
    def plot(self, ax, i_color):
        colors = ['cyan', 'red', 'blue', 'orange']
        c = colors[i_color]
        ax.plot(self.dict['X'], self.dict['Y'], color=c, lw=0.5)

### Deptho classes

In [141]:
class MultiBeadsDeptho:
   
    def __init__(self, I, fileName, scale):
        nz, ny, nx = I.shape[0], I.shape[1], I.shape[2]
        
        self.I = I
        self.fileName = fileName
        self.scale = scale
        self.I_labeled = np.zeros([nz, ny, nx])
        self.nz = nz
        self.ny = ny
        self.nx = nx
        self.listDepthos = []
        self.ND = 0
        self.NDvalid = 0
        self.z_max = 0
        self.threshold = 0.5*filters.threshold_otsu(I)
        self.summaryTable = pd.DataFrame({})
        
    def build_listDepthos(self, size = 'detect', plot = 0):
        maxVal = np.zeros(self.nz)
        for z in range(self.nz):
            maxVal[z] = np.max(self.I[z])
        z_max = np.argmax(maxVal)
        self.z_max = z_max
        I_max = self.I[z_max]
        threshold_z_max = filters.threshold_otsu(I_max)
        I_max_bin = (I_max > threshold_z_max)
        I_labeled, ND = measure.label(I_max_bin, return_num = True)
        self.I_labeled, self.ND = I_labeled, ND
        
        if plot >= 1:
            pStart, pStop = np.percentile(I[self.z_max], (1, 99))
            I_max_rescaled = exposure.rescale_intensity(I[self.z_max], in_range=(pStart, pStop))
            figI, axI = plt.subplots(1,1, figsize = (15,5))
            axI.imshow(I_max_rescaled, cmap = 'gray')
        
        # Building beadList
        iValid = 0
        for k in range(1,self.ND+1):
            ym0, xm0 = ndi.center_of_mass(I_max, I_labeled, k)
            bD = BeadDeptho(I, xm0, ym0, self.z_max, self.threshold, self.fileName, self.scale)
            bD.buildDeptho(size, plot)
            
            if bD.validBead:
                print('Job done for the bead in x0, y0 = {:.2f}, {:.2f}'.format(bD.xm0, bD.ym0))
                bD.build_zProfiles(plot)
                bD.iValid = iValid
                iValid += 1
                self.listDepthos.append(bD)
                self.NDvalid += 1
#                 if plot >= 1:
#                     fig, ax = plt.subplots(1,1, figsize = (8, 5))
#                     dV = bD.depthosDict['deptho_v']
#                     pStart, pStop = np.percentile(dV, (1, 99))
#                     dV_rescaled = exposure.rescale_intensity(dV, in_range=(pStart, pStop))
#                     ax.imshow(dV_rescaled, cmap='gray')
#                     ax.set_ylim([bD.zLast, bD.zFirst])
            else:
                print('Not acceptable bead in x0, y0 = {:.2f}, {:.2f}'.format(bD.xm0, bD.ym0))
            
            if plot >= 1:
                if bD.validBead:
                    axI.plot([bD.XYm[self.z_max, 0]], [bD.XYm[self.z_max, 1]], 'c+')
                    axI.text(bD.XYm[self.z_max, 0] + 5, bD.XYm[self.z_max, 1] - 5, str(bD.D), c = 'c')
                    bD.plotDeptho()
                else:
                    axI.plot([bD.xm0], [bD.ym0], 'r+')
            
            print('\n')  

In [134]:
# Building summaryTable
#         d = {'D' : [], 'x' : [], 'y' : []}
#         otherCols = [c for c in self.beadList[0].ZfocusDict.keys()]
#         for c in otherCols:
#             d[c] = []
#         for k in range(1,self.NDvalid+1):
#             bD = self.beadList[k-1]
#             d['D'].append(bD.D)
#             d['x'].append(bD.XYm[0,bD.z_max])
#             d['y'].append(bD.XYm[1,bD.z_max])
#             for c in otherCols:
#                 d[c].append(bD.ZfocusDict[c])        
#         self.summaryTable = pd.DataFrame(d)

In [154]:
class BeadDeptho:
    def __init__(self, I, xm0, ym0, z_max, threshold, fileName, scale):
        
        nz, ny, nx = I.shape[0], I.shape[1], I.shape[2]
        
        self.I = I
        self.nz = nz
        self.ny = ny
        self.nx = nx
        self.scale = scale
        self.xm0 = xm0
        self.ym0 = ym0
        self.fileName = fileName
        
        self.D = 0
        self.threshold = threshold
        self.I_cleanROI = np.array([])
#         self.cleanROI = np.zeros((self.nz, 4), dtype = int)
        self.XYm = np.zeros((self.nz, 2))
        self.validBead = True
        self.iValid = -1
        
        self.z_max = z_max
        self.validSlice = np.zeros(nz, dtype = bool)
        self.zFirst = 0
        self.zLast = nz
        self.validDepth = nz
        
        self.depthosDict = {}
        self.profileDict = {}
        self.ZfocusDict = {}
        
    def buildDeptho(self, size = 'detect', plot = 0):
        roughSize = np.floor(5*self.scale)
        x1, y1, x2, y2, validROI = getROI(roughSize, self.xm0, self.ym0, self.nx, self.ny)
        F_zmax = self.I[self.z_max, y1:y2, x1:x2]           
        F_zmax_bin = (F_zmax > self.threshold)
        F_zmax_lab, nBzmax = measure.label(F_zmax_bin, return_num = True)
        validBead =  validROI and (nBzmax == 1)
        self.validBead = validBead
        
        if validBead:            
            if size == 'detect':
                counts, binEdges = np.histogram(self.I[self.z_max,y1:y2,x1:x2].ravel(), bins=256)

                peaks, peaksProp = find_peaks(counts, height=100, threshold=None, distance=None, prominence=None, \
                                   width=None, wlen=None, rel_height=0.5, plateau_size=None)

                peakThreshVal = 1200

                if counts[peaks[0]] > peakThreshVal:
                    self.D = 4.5
                else:
                    self.D = 2.7
            
            elif size == 'M450':
                self.D = 4.5   
            elif size == 'M270':
                self.D = 2.7 
            else:
                print('invalid size input')
                return('invalid size input')
            
            self.XYm[self.z_max] = [self.xm0, self.ym0]
            
            previous_BoiXY = np.array([self.xm0, self.ym0])
            for z in range(self.z_max+1, self.nz):
                previous_BoiXY = self.trackBeadDeptho(z, previous_BoiXY)
                if not self.validSlice[z]:
                    break
            
            previous_BoiXY = np.array([self.xm0, self.ym0])
            for z in range(self.z_max-1, -1, -1):
                previous_BoiXY = self.trackBeadDeptho(z, previous_BoiXY)
                if not self.validSlice[z]:
                    break
        
#         print(self.XYm)
#         print(self.validSlice)
        
        zFirst = np.argmax(self.validSlice)
        zLast = self.nz - np.argmax(self.validSlice[::-1])

        roughSize = int(np.floor(1.2*self.D*self.scale))
        roughSize += 1 + roughSize%2
        roughCenter = int((roughSize+1)//2)
        cleanSize = int(np.floor(1*self.D*self.scale))
        cleanSize += 1 + cleanSize%2
        I_cleanROI = np.zeros([self.nz, cleanSize, cleanSize])
#         print(zFirst, zLast)
#         print(roughSize, roughCenter, cleanSize)

        try:
            for i in range(zFirst, zLast):
                xm0, ym0 = self.XYm[i,0], self.XYm[i,1]
                x1, y1, x2, y2, validBead = getROI(roughSize, xm0, ym0, self.nx, self.ny)
                xm1, ym1 = xm0-x1, ym0-y1
                I_roughRoi = self.I[i,y1:y2,x1:x2]
                translation = (xm1-roughCenter, ym1-roughCenter)

    #             if i == zFirst:
    #                 print('roughSize, roughCenter')
    #                 print(x2-x1, roughCenter)
    #                 print('xm0, ym0')
    #                 print(xm0, ym0)
    #                 print('x1, y1, x2, y2')
    #                 print(x1, y1, x2, y2)
    #                 print('xm1, ym1')
    #                 print(xm1, ym1)
    #                 print('I_roughRoi')
    #                 print(I_roughRoi)
    #                 print('translation')
    #                 print(translation)

                tform = transform.EuclideanTransform(rotation=0, \
                                                     translation = (xm1-roughCenter, ym1-roughCenter))
                I_tmp = transform.warp(I_roughRoi, tform, order = 1, preserve_range = True)

                I_cleanROI[i] = np.copy(I_tmp[roughCenter-cleanSize//2:roughCenter+cleanSize//2+1,\
                                              roughCenter-cleanSize//2:roughCenter+cleanSize//2+1])

    #             if i%30 == 0:
    #                 fig, ax = plt.subplots(1,3,figsize=(15,5))
    #                 ax[0].imshow(self.I[i], cmap='gray')
    #                 ax[0].plot([xm0], [ym0], 'b+')
    #                 ax[0].plot([x1,x1], [y1,y2], 'c--')
    #                 ax[0].plot([x1,x2], [y2,y2], 'c--')
    #                 ax[0].plot([x2,x2], [y1,y2], 'c--')
    #                 ax[0].plot([x1,x2], [y1,y1], 'c--')
    #                 mid = roughCenter
    #                 ax[1].imshow(I_roughRoi, cmap='gray')
    #                 ax[1].plot([0,2*mid],[mid, mid], 'r--', lw = 0.5)
    #                 ax[1].plot([mid, mid],[0,2*mid], 'r--', lw = 0.5)
    #                 ax[1].plot([xm1], [ym1], 'b+')
    #                 I_tmp_binary = (I_tmp > self.threshold)
    #                 y, x = ndi.center_of_mass(I_tmp, labels=I_tmp_binary, index=1)
    #                 ax[2].imshow(I_tmp, cmap='gray')
    #                 ax[2].plot([0,2*mid],[mid, mid], 'r--', lw = 0.5)
    #                 ax[2].plot([mid, mid],[0,2*mid], 'r--', lw = 0.5)
    #                 ax[2].plot([x], [y], 'b+')
    #                 fig.suptitle('Translation: ' + str(translation))
    #                 fig.show()

            self.zFirst = zFirst
            self.zLast = zLast
            self.validDepth = zLast-zFirst
            self.I_cleanROI = I_cleanROI.astype(np.uint16)

            # VISUALISE
            if plot >= 2:
                for i in range(zFirst, zLast, 50):
                    self.plotROI(i)

        except:
            print('Error for the bead in x0, y0 = {:.2f}, {:.2f}'.format(self.xm0, self.ym0))
              
    def trackBeadDeptho(self, z, previous_BoiXY):
        self.validSlice[z] = True
        
        roughSize = np.floor(2*self.D*self.scale)
        x1, y1, x2, y2, validROI = getROI(roughSize, previous_BoiXY[0], previous_BoiXY[1], self.nx, self.ny)
    
        if not validROI:
            self.validSlice[z] = False
#             self.validBead = False
            return(previous_BoiXY)

        else:
            F_z = self.I[z, y1:y2, x1:x2]
            previous_BoiXY_ROI = previous_BoiXY - np.array([x1, y1])                  
            F_z_bin = (F_z > self.threshold)
            F_z_bin_fh = ndi.binary_fill_holes(F_z_bin).astype(int)
            F_z_lab, nB_ROI = measure.label(F_z_bin_fh, return_num = True)

            if nB_ROI == 0:
                self.validSlice[z] = False
                return(previous_BoiXY)

            else: # (if nB_ROI > 0)
                props = measure.regionprops(F_z_lab)
                listValidLabels = []
                for k in range(1, nB_ROI+1):
                    try:
                        bb = props[k-1].bbox
                        validObj = not (min(bb) == 0 or bb[2] == self.ny or bb[3] == self.nx) # Remove objects touching the edges of the frame
                        A = props[k-1].area
                        P = props[k-1].perimeter
                        Circ = (4 * np.pi * A) / (P * P)
                        validObj = validObj and A >= 50 and Circ >= 0.75 # Area and circularity criterion
                    except:
                        validObj = False
                    if validObj:
                        listValidLabels.append(k)

                if len(listValidLabels) == 0:
                    self.validSlice[z] = False
                    return(previous_BoiXY)

                else:
                    centroidsYX_list = [ndi.center_of_mass(F_z, labels=F_z_lab, index=i) for i in listValidLabels]
                    centroidsXY = np.array([[centroidsYX_list[k][1], centroidsYX_list[k][0]] for k in range(len(centroidsYX_list))])
                    M = compute_cost_matrix(np.array([previous_BoiXY_ROI]), centroidsXY)
                    i_best = np.argmin(M[0])
                    cost_best = M[0][i_best]**0.5

                    if cost_best/SCALE_100X > 0.5:
                        fig, ax = plt.subplots(1,1, figsize = (8,8))
                        pStart, pStop = np.percentile(F_z, (1, 99))
                        ax.imshow(F_z, cmap = 'gray', vmin = pStart, vmax = pStop)
                        ax.plot(centroidsXY[:,0], centroidsXY[:,1], c='orange', marker='o')
                        fig.show()
                        mngr = plt.get_current_fig_manager()
                        mngr.window.setGeometry(700,100,1000, 800)
                        QA = pyautogui.confirm(
                            text='Can you point the bead of interest\nin the image ' + str(z + 1) + '?',
                            title='', 
                            buttons=['No', 'Yes', 'Abort Mission!'])
                        # According to the question's answer:
                        if QA == 'Yes':
                            ui = plt.ginput(1, timeout=0)
                            uiXY = ui2array(ui)
                            fig = plt.gcf()
                            plt.close(fig)
                            M = compute_cost_matrix(uiXY,centroidsXY)
                            i_best = np.argmin(M[0])
                        elif QA == 'No':
                            fig = plt.gcf()
                            plt.close(fig)
                            self.validSlice[z] = False
                        elif QA == 'Abort Mission!':
                            fig = plt.gcf()
                            plt.close(fig)
                            return('Abort!')
                            
                    if self.validSlice[z]:
                        BoiXY = np.array(centroidsXY[i_best,:]) + np.array([x1, y1])
                        self.XYm[z] = BoiXY
                        # self.validSlice[z] = True (already the case)
                        return(BoiXY)
                    else:
                        return(previous_BoiXY)
    
    def build_zProfiles(self, plot = 0):
        side_ROI = self.I_cleanROI.shape[1]
        mid_ROI = side_ROI//2
        nbPixToAvg = 3 # Have to be an odd number
        deptho_v = np.zeros([self.nz, side_ROI], dtype = np.float64)
        deptho_h = np.zeros([self.nz, side_ROI], dtype = np.float64)
        
        for z in range(self.zFirst, self.zLast):
            templine = side_ROI
            deptho_v[z] = self.I_cleanROI[z,:,mid_ROI] * (1/nbPixToAvg)
#             print(self.I_cleanROI[z,:,mid_ROI])
#             print(1/nbPixToAvg)
#             print(self.deptho_v[z])
            deptho_h[z] = self.I_cleanROI[z,mid_ROI,:] * (1/nbPixToAvg)
    
            for i in range(1, 1 + nbPixToAvg//2):
#                 print(self.I_cleanROI[z,:,mid_ROI - i] * (1/nbPixToAvg) )
                deptho_v[z] += self.I_cleanROI[z,:,mid_ROI - i] * (1/nbPixToAvg) 
                deptho_v[z] += self.I_cleanROI[z,:,mid_ROI + i] * (1/nbPixToAvg)
                
                deptho_h[z] += self.I_cleanROI[z,mid_ROI - i,:] * (1/nbPixToAvg) 
                deptho_h[z] += self.I_cleanROI[z,mid_ROI + i,:] * (1/nbPixToAvg)
          
        deptho_v = deptho_v.astype(np.uint16)
        deptho_h = deptho_h.astype(np.uint16)
        
        self.depthosDict['deptho_v'] = deptho_v
        self.depthosDict['deptho_h'] = deptho_h
        
        
        # 3D caracterisation
        I_binary = np.zeros([self.I_cleanROI.shape[0], self.I_cleanROI.shape[1], self.I_cleanROI.shape[2]])
        I_binary[self.zFirst:self.zLast] = (self.I_cleanROI[self.zFirst:self.zLast] > self.threshold)
        Zm3D, Ym3D, Xm3D = ndi.center_of_mass(self.I_cleanROI, labels=I_binary, index=1)
        self.ZfocusDict['Zm3D'] = Zm3D
        
        
        # Raw profiles
        
        Z = np.array([z for z in range(self.I_cleanROI.shape[0])])
        intensity_tot = np.array([np.sum(self.I_cleanROI[z][I_binary[z].astype(bool)])/(1+np.sum(I_binary[z])) for z in range(self.I_cleanROI.shape[0])]).astype(np.float64)
        intensity_v = np.array([np.sum(deptho_v[z,:])/side_ROI for z in range(deptho_v.shape[0])]).astype(np.float64)
        intensity_h = np.array([np.sum(deptho_h[z,:])/side_ROI for z in range(deptho_h.shape[0])]).astype(np.float64)
        
        Zm_v, Zm_h, Zm_tot = np.argmax(intensity_v), np.argmax(intensity_h), np.argmax(intensity_tot)
        
        self.profileDict['intensity_v'] = intensity_v
        self.profileDict['intensity_h'] = intensity_h
        self.profileDict['intensity_tot'] = intensity_tot
        self.ZfocusDict['Zm_v'] = Zm_v
        self.ZfocusDict['Zm_h'] = Zm_h
        self.ZfocusDict['Zm_tot'] = Zm_tot


        # Smoothed profiles
        
        Z_hd = np.arange(0,self.I_cleanROI.shape[0],0.2)
        intensity_v_hd = np.interp(Z_hd, Z, intensity_v)
        intensity_h_hd = np.interp(Z_hd, Z, intensity_h)
        intensity_tot_hd = np.interp(Z_hd, Z, intensity_tot)
        intensity_v_smooth = savgol_filter(intensity_v_hd, 101, 7)
        intensity_h_smooth = savgol_filter(intensity_h_hd, 101, 7)
        intensity_tot_smooth = savgol_filter(intensity_tot_hd, 101, 7)
        
        Zm_v_hd, Zm_h_hd, Zm_tot_hd = Z_hd[np.argmax(intensity_v_smooth)], Z_hd[np.argmax(intensity_h_smooth)], Z_hd[np.argmax(intensity_tot_smooth)]
        
        self.profileDict['intensity_v_smooth'] = intensity_v_smooth 
        self.profileDict['intensity_h_smooth'] = intensity_h_smooth
        self.profileDict['intensity_tot_smooth'] = intensity_tot_smooth
        self.ZfocusDict['Zm_v_hd'] = Zm_v_hd 
        self.ZfocusDict['Zm_h_hd'] = Zm_h_hd
        self.ZfocusDict['Zm_tot_hd'] = Zm_tot_hd
        
        # VISUALISE
        if plot >= 2:
            self.plotProfiles()
            

    def plotXYm(self):
        fig, ax = plt.subplots(1,1)
        pStart, pStop = np.percentile(self.I[self.z_max], (1, 99))
        ax.imshow(self.I[self.z_max], cmap = 'gray', vmin = pStart, vmax = pStop)
        ax.plot(self.XYm[self.validSlice,0],self.XYm[self.validSlice,1],'r-')
        fig.show()
        
    def plotROI(self, i = 'auto'):
        if i == 'auto':
            i = self.z_max
        
        fig, ax = plt.subplots(1,3, figsize = (16,4))
        
        xm, ym = np.mean(self.XYm[self.validSlice,0]),  np.mean(self.XYm[self.validSlice,1])
        ROIsize_x = self.D*1.25*self.scale + (max(self.XYm[self.validSlice,0])-min(self.XYm[self.validSlice,0]))
        ROIsize_y = self.D*1.25*self.scale + (max(self.XYm[self.validSlice,1])-min(self.XYm[self.validSlice,1]))
        x1_ROI, y1_ROI, x2_ROI, y2_ROI = int(xm - ROIsize_x//2), int(ym - ROIsize_y//2), int(xm + ROIsize_x//2), int(ym + ROIsize_y//2)        

        pStart, pStop = np.percentile(self.I[i], (1, 99))
        ax[0].imshow(self.I[i], cmap = 'gray', vmin = pStart, vmax = pStop)
        ax[0].plot([x1_ROI,x1_ROI], [y1_ROI,y2_ROI], 'c--')
        ax[0].plot([x1_ROI,x2_ROI], [y2_ROI,y2_ROI], 'c--')
        ax[0].plot([x2_ROI,x2_ROI], [y1_ROI,y2_ROI], 'c--')
        ax[0].plot([x1_ROI,x2_ROI], [y1_ROI,y1_ROI], 'c--')

        I_ROI = self.I[i,y1_ROI:y2_ROI,x1_ROI:x2_ROI] 
        pStart, pStop = np.percentile(I_ROI, (1, 99))
        ax[1].imshow(I_ROI, cmap = 'gray', vmin = pStart, vmax = pStop)
        ax[1].plot(self.XYm[self.validSlice,0]-x1_ROI, self.XYm[self.validSlice,1]-y1_ROI, 'r-', lw=0.75)
        ax[1].plot(self.XYm[i,0]-x1_ROI, self.XYm[i,1]-y1_ROI, 'b+', lw=0.75)
        
        pStart, pStop = np.percentile(self.I_cleanROI[i], (1, 99))
        mid = self.I_cleanROI[i].shape[0]//2
        I_cleanROI_binary = (self.I_cleanROI[i] > self.threshold)
        y, x = ndi.center_of_mass(self.I_cleanROI[i], labels=I_cleanROI_binary, index=1)
        ax[2].imshow(self.I_cleanROI[i], cmap = 'gray', vmin = pStart, vmax = pStop)
        ax[2].plot([0,2*mid],[mid, mid], 'r--', lw = 0.5)
        ax[2].plot([mid, mid],[0,2*mid], 'r--', lw = 0.5)
        ax[2].plot([x],[y], 'b+')
        fig.show()

    def plotProfiles(self):
        Z = np.array([z for z in range(self.I_cleanROI.shape[0])])
        Z_hd = np.arange(0,self.I_cleanROI.shape[0],0.2)
        intensity_v = self.profileDict['intensity_v']
        intensity_h = self.profileDict['intensity_h']
        intensity_tot = self.profileDict['intensity_tot']
        Zm_v = self.ZfocusDict['Zm_v']
        Zm_h = self.ZfocusDict['Zm_h']
        Zm_tot = self.ZfocusDict['Zm_tot']
        intensity_v_smooth = self.profileDict['intensity_v_smooth']
        intensity_h_smooth = self.profileDict['intensity_h_smooth']
        intensity_tot_smooth = self.profileDict['intensity_tot_smooth']
        Zm_v_hd = self.ZfocusDict['Zm_v_hd']
        Zm_h_hd = self.ZfocusDict['Zm_h_hd']
        Zm_tot_hd = self.ZfocusDict['Zm_tot_hd']
        
#             fig, ax = plt.subplots(1,3, figsize = (12, 4))
#             ax[0].plot(Z, intensity_v)
#             ax[1].plot(Z, intensity_h)
#             ax[2].plot(Z, (intensity_tot))
#             ax[0].plot([Zm_v, Zm_v], [0, ax[0].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_v = {:.2f}'.format(Zm_v))
#             ax[1].plot([Zm_h, Zm_h], [0, ax[1].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_h = {:.2f}'.format(Zm_h))
#             ax[2].plot([Zm_tot, Zm_tot], [0, ax[2].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_tot = {:.2f}'.format(Zm_tot))
#             ax[0].legend(loc = 'lower right')
#             ax[1].legend(loc = 'lower right')
#             ax[2].legend(loc = 'lower right')
        
        fig, ax = plt.subplots(1,3, figsize = (12, 4))
        ax[0].plot(Z, intensity_v, 'b-')
        ax[1].plot(Z, intensity_h, 'b-')
        ax[2].plot(Z, (intensity_tot), 'b-')
        ax[0].plot(Z_hd, intensity_v_smooth, 'k--')
        ax[1].plot(Z_hd, intensity_h_smooth, 'k--')
        ax[2].plot(Z_hd, intensity_tot_smooth, 'k--')
        ax[0].plot([Zm_v_hd, Zm_v_hd], [0, ax[0].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_v_hd = {:.2f}'.format(Zm_v_hd))
        ax[1].plot([Zm_h_hd, Zm_h_hd], [0, ax[1].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_h_hd = {:.2f}'.format(Zm_h_hd))
        ax[2].plot([Zm_tot_hd, Zm_tot_hd], [0, ax[2].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_tot_hd = {:.2f}'.format(Zm_tot_hd))
        ax[0].legend(loc = 'lower right')
        ax[1].legend(loc = 'lower right')
        ax[2].legend(loc = 'lower right')

        #         print('Zm_v = {:.2f}, Zm_h = {:.2f}, Zm_tot = {:.2f}'\
        #               .format(Zm_v, Zm_h, Zm_tot))
        #         print('Zm_v_hd = {:.2f}, Zm_h_hd = {:.2f}, Zm_tot_hd = {:.2f}'\
        #               .format(Zm_v_hd, Zm_h_hd, Zm_tot_hd))
        
        fig.show()
        
    def plotDeptho(self, d = 'v'):
        fig, ax = plt.subplots(1,1, figsize = (4, 6))
        if d == 'v':
            D = self.depthosDict['deptho_v']
            z_focus = self.ZfocusDict['Zm_v_hd']
        elif d == 'h':
            D = self.depthosDict['deptho_h']
            z_focus = self.ZfocusDict['Zm_h_hd']
        ny, nx = D.shape[0], D.shape[1]
        pStart, pStop = np.percentile(D, (1, 99))
        ax.imshow(D, cmap='gray', vmin = pStart, vmax = pStop)
        ax.plot([0, nx], [self.zFirst, self.zFirst], 'r--')
        ax.text(nx//2, self.zFirst - 10, str(self.zFirst), c = 'r')
        ax.plot([0, nx], [self.zLast, self.zLast], 'r--')
        ax.text(nx//2, self.zLast - 10, str(self.zLast), c = 'r')
        ax.plot([0, nx], [z_focus, z_focus], 'c--')
        ax.text(nx//2, z_focus - 10, str(z_focus), c = 'c')
        fig.suptitle('File ' + self.fileName + ' - Bead ' + str(self.iValid))
        fig.show()

In [159]:
def depthoMaker(dirPath, scale, d = 'v'):
    listDepthosFileNames = [f for f in os.listdir(dirPath) if (os.path.isfile(os.path.join(dirPath, f)) and f.endswith(".tif"))]
    listMBDepthos = []
    listSizes = []
    dirMaxValidDepth
    for f in listDepthosFileNames:
        filePath = os.path.join(dirPath, f)
        I = io.imread(filePath)
        mbd = MultiBeadsDeptho(I, f[:-4], scale)
        mbd.build_listDepthos(size = 'detect', plot = 1)
        listMBDepthos.append(mbd)
        for bD in mbd.listDepthos:
            if bD.D not in listSizes:
                listSizes.append(bD.D)
        print(listSizes)
# TBC

In [156]:
# testDataFolder = 'C://Users//JosephVermeil//Desktop//test_DepthoMaking'
testDataFolder = 'C://Users//JosephVermeil//Desktop//test_AnalyseDepthos'
listDepthos = [f for f in os.listdir(testDataFolder) if (os.path.isfile(os.path.join(testDataFolder, f)) and f.endswith(".tif"))]
listDepthos

['multiDeptho10.tif', 'multiDeptho9.tif']

In [157]:
f = listDepthos[1]
filePath = os.path.join(testDataFolder, f)
I = io.imread(filePath)
mbd = MultiBeadsDeptho(I, f[:-4], SCALE_100X)
mbd.build_listDepthos(size = 'detect', plot = 1)
# listBeads = mbd.get_beadList()

BUG ! for the bead in x0, y0 = 2031.70, 21.24
Not acceptable bead in x0, y0 = 2031.70, 21.24


Job done for the bead in x0, y0 = 1903.86, 115.13


BUG ! for the bead in x0, y0 = 1256.49, 122.10
Not acceptable bead in x0, y0 = 1256.49, 122.10


BUG ! for the bead in x0, y0 = 1296.29, 135.56
Not acceptable bead in x0, y0 = 1296.29, 135.56




C:\Users\JosephVermeil\anaconda3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: divide by zero encountered in double_scalars


Job done for the bead in x0, y0 = 669.32, 218.49


Job done for the bead in x0, y0 = 1443.07, 275.02


Job done for the bead in x0, y0 = 982.62, 275.52


Job done for the bead in x0, y0 = 1318.99, 417.28


Job done for the bead in x0, y0 = 1027.75, 503.07


BUG ! for the bead in x0, y0 = 82.49, 579.09
Not acceptable bead in x0, y0 = 82.49, 579.09


BUG ! for the bead in x0, y0 = 1692.01, 586.24
Not acceptable bead in x0, y0 = 1692.01, 586.24


BUG ! for the bead in x0, y0 = 150.52, 597.67
Not acceptable bead in x0, y0 = 150.52, 597.67




In [ ]:
print(mbd.threshold)
for bD in mbd.listDepthos:
    bD.plotXYm()

In [109]:
for bD in mbd.listDepthos:
    fig, ax = plt.subplots(1,2, figsize = (12, 4))
    Z = np.array([z for z in range(bD.I_cleanROI.shape[0])])
    deptho_v = bD.depthosDict['deptho_v']
    deptho_h = bD.depthosDict['deptho_h']
    ax[0].imshow(deptho_v, cmap='gray')
    ax[1].imshow(deptho_h, cmap='gray')
    fig.show()

In [19]:
bD = mbd.listDepthos[0]
for z in range(bD.zFirst,bD.zLast):
    if z%20==0:
        fig, ax = plt.subplots(1,1)
        ax.imshow(bD.I_cleanROI[z], cmap='gray')
#         ax.plot(bD.XYm)
        fig.show()

## Main

In [ ]:
mainDataDir = 'D://MagneticPincherData'
# mainDataDir = 'C://Users//josep//Desktop//TestData_BeadTracker'
rawDataDir = os.path.join(mainDataDir, 'Raw')
interDataDir = os.path.join(mainDataDir, 'Intermediate')
figureDir = os.path.join(mainDataDir, 'Figures')
dateTest1 = '21-04-27'
dateTest2 = '21.04.27'
# 21-04-27_M1_P1_C4_R40_disc20um_wFluo
# imagesToAnalyse.append(os.path.join(rd, f))

In [ ]:
def main():
    
    start = time.time()
    
    dates = '21.01.21'
#     dates = '21.04.27'
#     manip = 'M1_P1_C4'
#     manipID = '21-04-27_M1'
    
    manips = 1
    wells = 1
    cells = 1
    
    
    
    # 0. Load different data sources & Preprocess : fluo, black images, sort slices (ct/ramp ; down/middle/up)
    # Make list of files to analyse
    imagesToAnalyse = []
    imagesToAnalyse_Paths = []
    if not isinstance(dates, str):
        rawDirList = [os.path.join(rawDataDir, d) for d in dates]
    else:
        rawDirList = [os.path.join(rawDataDir, dates)]
    for rd in rawDirList:
        fileList = os.listdir(rd)
        for f in fileList:
            if isFileOfInterest(f, manips, wells, cells):
                imagesToAnalyse.append(f)
                imagesToAnalyse_Paths.append(os.path.join(rd, f))          
    
    # Begining of the MAIN LOOP
    for i in range(len(imagesToAnalyse)): 
        f, fP = imagesToAnalyse[i], imagesToAnalyse_Paths[i]
        manipID = findInfosInFileName(f, 'manipID')
        cellID = findInfosInFileName(f, 'cellID')
        
        # Load exp data
        if manipID not in expDf['manipID'].values:
            print('Error! No experimental data found for: ' + manipID)
            bug
        else:
            expDf_line = expDf.loc[expDf['manipID'] == manipID]
            manipDict = {}
            for c in expDf_line.columns.values:
                manipDict[c] = expDf_line[c].values[0]
    
        # Load image and init PTL
        I = io.imread(fP) # Approx 0.5s per image
        PTL = PincherTimeLapse(I, cellID, manipDict, NB = 2)
    
        # Load field file
        fieldFilePath = fP[:-4] + '_Field.txt'
        fieldCols = ['B_set', 'T_abs', 'B', 'Z']
        fieldDf = pd.read_csv(fieldFilePath, sep = '\t', names = fieldCols)
        
        # Check if a log file exists and load it if required
        logFilePath = fP[:-4] + '_Log.txt'
        logFileImported = False
        if os.path.isfile(logFilePath):
            PTL.importLog(logFilePath)
            PTL.dictLog['UILog'] = PTL.dictLog['UILog'].astype(str)
            logFileImported = True
        
        # Detect fluo & black images
        PTL.checkIfBlackFrames()
        PTL.saveFluoAside()
        if not logFileImported:
            pass # I may change the organization of this part later
        
        # Sort slices
        if not logFileImported:
            PTL.determineFramesStatus()
        PTL.saveLog(display = True, save = (not logFileImported), path = logFilePath)
        
        # Determine global threshold
        PTL.computeThreshold() # Approx 3s per image
#         PTL.testThresholding()
        
        # Create list of Frame objects
        PTL.makeFramesList()
    
    
    # 1. Detect beads
        # Check if a _Results.txt exists and import it if so
        resFilePath = fP[:-4] + '_ResultsPY.txt'
        resFileImported = False
        if os.path.isfile(resFilePath):
            PTL.importBeadsDetectResult(resFilePath)
            resFileImported = True
    
        # Dectect the beads and create the BeadsDetectResult dataframe [if no file has been loaded before] 
        # OR input the results in each Frame objects [if the results have been loaded at the previous step]
        PTL.detectBeads(resFileImported, display = 1)
        
        # Save the new results if necessary
        if not resFileImported:
            PTL.saveBeadsDetectResult(path=resFilePath)

        
    # 2. Make trajectories for beads of interest
        # Display the first image to ask for the 2 beads of interest
        issue = PTL.buildTrajectories()
        if issue == 'Bug':
            continue
        else:
            pass
        PTL.saveLog(display = 2, save = True, path = logFilePath)
        
        # 
    
    # 3. Qualify - Detect boi sizes and neighbours
        # Detect Boi sizes in the first image and propagate it across the trajectories
        PTL.listFrames[0].detectDiameter(plot = 0)
        for iB in range(PTL.NB):
            traj = PTL.listTrajectories[iB]
            B0 = traj.dict['Bead'][0]
            D = B0.D
            traj.D = D
            for B in traj.dict['Bead']:
                B.D = D
        
        # Detect neighbours in every first image of loop
        iL = 0
        for iB in range(PTL.NB):
            traj = PTL.listTrajectories[iB]
            neighbours = np.array(['' for i in range(traj.nT)], dtype = '<U12')
            current_neighbours = ''
            for i in range(traj.nT):
                iS = traj.dict['iS'][i]
                if iS >= iL*PTL.loop_totalSize:
                    # Redo the detection
                    iF = traj.dict['iF'][i]
                    bead = traj.dict['Bead'][i]
                    frame = PTL.listFrames[iF]
                    current_neighbours = frame.detectNeighbours(bead)
                    neighbours[i] = current_neighbours
                    #
                    iL += 1
                else:
                    # Just take the previous neighbour situation
                    neighbours[i] = current_neighbours
                    
            traj.dict['neighbours'] = neighbours
    
        
    # 4. Compute dz
        # Import depthographs
        
        
        # Compute z for each traj
    
    
    
    # 5. Define pairs and compute dx, dy
    
        if PTL.NB == 2:
            traj1 = PTL.listTrajectories[0]
            traj2 = PTL.listTrajectories[1]
            nT = traj1.nT
            
            # Create a dict to prepare the export of the results
            timeSeries = {
                'idxCompression' : np.zeros(nT), \
                'T' : np.zeros(nT), \
                'Tabs' : np.zeros(nT), \
                'B' : np.zeros(nT), \
                'F' : np.zeros(nT), \
                'dx' : np.zeros(nT), \
                'dy' : np.zeros(nT), \
                'dz' : np.zeros(nT), \
                'D2' : np.zeros(nT), \
                'D3' : np.zeros(nT)
            }

            # Input common values:
            T0 = fieldDf['T_abs'].values[0]/1000
            timeSeries['idxCompression'] = traj1.dict['idxCompression']
            timeSeries['Tabs'] = (fieldDf['T_abs'][traj1.dict['iField']])/1000
            timeSeries['T'] = timeSeries['Tabs'].values - T0*np.ones(nT)
            timeSeries['B'] = fieldDf['B'][traj1.dict['iField']]

            # Compute distances
            timeSeries['dx'] = traj2.dict['X'] - traj1.dict['X']
            timeSeries['dy'] = traj2.dict['Y'] - traj1.dict['Y']
            timeSeries['D2'] = (timeSeries['dx']**2 +  timeSeries['dy']**2)**0.5
            
            timeSeries_DF = pd.DataFrame(timeSeries)
            print('\n\n* timeSeries:\n')
            print(timeSeries_DF)
            
    # 6. Compute forces
    # 7. Export the results
    
    print('\nTotal time:')
    print(time.time()-start)
    return(PTL)

In [ ]:
PTL = main()

In [ ]:
PTL.listTrajectories[0].to_df()

In [ ]:
frame = PTL.listFrames[-5].show()
ax = plt.gca()
for i in range(PTL.NB):
    PTL.listTrajectories[i].plot(ax, i)
plt.show()

In [ ]:
traj = PTL.listTrajectories[1]
traj.to_df()

In [ ]:
traj = PTL.listTrajectories[1]
a = np.sum(np.array(traj.dict['idxCompression']) == 1)
traj.dict['bestStd'][:]

In [ ]:
traj = PTL.listTrajectories[1]
traj.dict['iField'][:]

In [ ]:
traj = PTL.listTrajectories[1]
np.array(traj.dict['idxRamp']) == 1

In [ ]:
# PTL.dictLog['UILog'] = PTL.dictLog['UILog'].astype(str)
a = PTL.dictLog['UILog'].astype(str)
a

In [ ]:
plt.close('all')

In [ ]:
# fp = 'D://MagneticPincherData//Raw//21.04.27//21-04-27_M1_P1_C4_R40_disc20um_wFluo.tif'
# fp = "C://Users//JosephVermeil//Desktop//Fichier 3BiocheVsPhy@2x.png"
# fp = "C://Users//JosephVermeil//Desktop//21-04-27_M1_P1_C4_R40_disc20um_wFluo.tif"
# I = io.imread(fp)
# I

In [ ]:
a = ['a']
b = 'a'
len(a[0][0])
len(b[0][0])
isinstance(b, str)